In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

# import matplotlib.tight_bbox as tight_bbox
import matplotlib.animation as animation
import matplotlib.pylab as plt

from IPython.display import HTML
from scipy.ndimage import zoom

import numpy as np
import pandas as pd
import trimesh
from scipy.ndimage import binary_dilation, binary_erosion

import glob
import os


def pad_to_square(image):
    H, W, S, T = image.shape
    if H == W:
        return image  # Already square​

    # Calculate padding
    if H < W:
        pad_top = (W - H) // 2
        pad_bottom = (W - H) - pad_top
        padding = ((pad_top, pad_bottom), (0, 0), (0, 0), (0, 0))
    else:  # W < H
        pad_left = (H - W) // 2
        pad_right = (H - W) - pad_left
        padding = ((0, 0), (pad_left, pad_right), (0, 0), (0, 0))

    # Apply padding
    padded_image = np.pad(image, padding, mode='constant', constant_values=0)
    return padded_image


def load_nii(nii_path):
    file = nib.load(nii_path)
    data = file.get_fdata()
    return data


def make_3d_video(masks, patient):
    fig = plt.figure(figsize = (3,3), facecolor='none', edgecolor='none', alpha=0)
    ax = fig.add_subplot(111, projection='3d')
    ax.axis('off')
    frames = []
    timeslices = masks.shape[-2]
    skips = 1
    for time in range(0, timeslices, skips):
        mask = masks[:,:,:,time,-1].copy() # blood pool 
        # mask = extract_largest_component(mask) # post processing
        mask = binary_dilation(mask,  iterations = 3) # for tidying
        mask = binary_erosion(mask, iterations = 3) # for tidying
        mesh = trimesh.voxel.ops.matrix_to_marching_cubes(mask)  # convert to mask
        mesh = trimesh.smoothing.filter_taubin(mesh, iterations = 50, lamb=1) # smoothing
        vertices = mesh.vertices
        faces = mesh.faces
        frame = ax.plot_trisurf(vertices[:,0], vertices[:,1], vertices[:,2], triangles=faces, color = 'red')
        
        mask = np.sum(masks[:,:,:,time,-2:], -1).copy() # myocardium 
        # mask = extract_largest_component(mask) # post processing
        mask = binary_dilation(mask,  iterations = 3) # for tidying
        mask = binary_erosion(mask, iterations = 3) # for tidying
        mesh = trimesh.voxel.ops.matrix_to_marching_cubes(mask)  # convert to mask
        mesh = trimesh.smoothing.filter_taubin(mesh, iterations = 50, lamb=1) # smoothing
        vertices = mesh.vertices
        faces = mesh.faces
        frame2 = ax.plot_trisurf(vertices[:,0], vertices[:,1], vertices[:,2], triangles=faces, color = '#0098ff', alpha = 0.2)

        ax.dist = 8
        frames.append([frame, frame2])      
    fig.tight_layout()
    ani = animation.ArtistAnimation(fig, frames)
    ani.save(f'3dplot/{patient}.gif', fps = int(timeslices/skips))
    plt.close()


def make_2d_video(image, mask, patient):
    position = image.shape[2]
    timesteps = image.shape[3]
    num_tiles = position

    grid_rows = int(np.sqrt(num_tiles) + 0.5)  # Round.
    grid_cols = (num_tiles + grid_rows - 1) // grid_rows     # Ceil.


    row_cols = np.ceil(np.sqrt(position)) 
    fig, axes = plt.subplots(grid_rows,grid_cols,figsize = (grid_cols*3,grid_rows*3))

    frames = []
    for time in range(timesteps):
        ttl = plt.text(0.5, 1.01, f'timestep = {time + 1}/{timesteps}', horizontalalignment='center', verticalalignment='bottom', transform=axes[0,0].transAxes, fontsize="large")
        artists = [ttl]
        for row, col in np.ndindex(grid_rows, grid_cols): 
            axes[row, col].axis('off')
            axes[row, col].patch.set_facecolor('white')
            pos = row * grid_cols + col
            if pos < position:
                artists.append(axes[row, col].imshow(image[:,:,pos, time], cmap = 'gray', vmin = np.min(image), vmax = np.max(image)))
                artists.append(axes[row, col].imshow(mask[:,:,pos, time, 1], alpha = mask[:,:,pos, time, 1] * 0.8, cmap = 'Blues'))
                artists.append(axes[row, col].imshow(mask[:,:,pos, time, 2], alpha = mask[:,:,pos, time, 2] * 0.8, cmap = 'jet'))
        frames.append(artists)
    bbox_inches = fig.get_tightbbox(fig.canvas.get_renderer())
    bbox_inches = bbox_inches.padded(0.1)
    tight_bbox.adjust_bbox(fig, bbox_inches)
    fig.set_size_inches(bbox_inches.width, bbox_inches.height)
    ani = animation.ArtistAnimation(fig, frames)
    ani.save(f'2dplot/{patient}.gif', fps=round(timesteps/2))
    plt.close()

def find_crop_box(mask, crop_factor):
    '''
    Calculated a bounding box that contains the masks inside
    '''
    mask = np.eye(3)[mask] 
    mask = np.sum(mask[...,1:], axis = (2,3,4))


    x = np.sum(mask, axis = 1)
    y = np.sum(mask, axis = 0)

    top = np.min(np.nonzero(x)) - 1
    bottom = np.max(np.nonzero(x)) + 1

    left = np.min(np.nonzero(y)) - 1
    right = np.max(np.nonzero(y)) + 1
    if abs(right - left) > abs(top - bottom):
            largest_side = abs(right - left)
    else:
        largest_side = abs(top - bottom)
    x_mid = round((left + right)/2)
    y_mid = round((top + bottom)/2)
    half_largest_side = round(largest_side * crop_factor/2)
    x_max, x_min = round(x_mid + half_largest_side), round(x_mid - half_largest_side)
    y_max, y_min = round(y_mid + half_largest_side), round(y_mid - half_largest_side)
    if x_min < 0:
        x_max -= x_min 
        x_min = 0
        
    if y_min < 0:
        y_max -= y_min
        y_min = 0 
    
    return [x_min, y_min, x_max, y_max]




# Load the NIfTI file
path = 'cine_sax_compressed_derivatives'
output_path = 'voxel/clean'

done = ['2.16.840.1.114151.1040466990330684466720442202860436476150210802', '1.2.528.1.1003.1.12306874907342000056166814777649.2', '1.2.840.114350.2.277.2.798268.2.406274773.1', '1.3.6.1.4.1.53684.1.1.2.1671342546.12108.1627063752.57184', '1.3.12.2.1107.5.8.15.132618.30000021062909173058800000012', '1.3.6.1.4.1.53684.1.1.2.1671342546.8128.1621432757.743519', '2.16.840.1.114151.4.25.40576.4995.6361118', '1.3.6.1.4.1.53684.1.1.2.3897358757.4924.1619544600.383348', '1.2.528.1.1003.2.10901914927613142541511812074232.2', '1.2.840.113619.2.25.4.2147483647.1659108693.778', '2.16.840.1.114151.4.25.42778.6563.12373853', '1.3.12.2.1107.5.8.15.132618.30000021071607425791500004404', '1.2.840.113619.2.182.12819322223169136.1363971708.388181', '1.3.6.1.4.1.53684.1.1.2.3897358757.14268.1626708633.215421', '1.3.6.1.4.1.53684.1.1.2.3897358757.6976.1620222547.350219', '1.3.6.1.4.1.53684.1.1.2.1671342546.8128.1621258825.263840', '1.3.6.1.4.1.53684.1.1.2.1671342546.11192.1624643136.419061', '1.3.6.1.4.1.53684.1.1.2.3897358757.1860.1617893521.1017219', '1.3.6.1.4.1.53684.1.1.2.1887816670.1968.1659366948.2595485', '1.2.840.114350.2.331.2.798268.2.271810545.1', '1.3.6.1.4.1.53684.1.1.2.3897358757.1880.1610124640.32820', '1.2.840.113619.2.182.108086170110251.1561721516.741144', '1.2.528.1.1003.1.12230536119578000056166814777649.2', '1.2.840.113745.101000.206701.42396.3383.36106795', '1.3.6.1.4.1.53684.1.1.2.1671342546.8364.1626281189.1366823', '2.16.840.1.114151.4.25.42539.6149.11593678', '2.16.840.1.114151.218662426160992186304091696922497798390220812', '1.3.6.1.4.1.53684.1.1.2.1887816670.1920.1631635957.1082591', '1.2.840.113845.11.1000000001785349915.20200803103851.1231167', '1.3.6.1.4.1.53684.1.1.2.1671342546.8128.1621008563.152866', '1.2.840.114350.2.331.2.798268.2.668950564.1', '1.3.6.1.4.1.53684.1.1.2.3897358757.1880.1610465595.344101', '2.16.840.1.114151.2.2.1.42688.3444.27701323', '1.2.840.113711.999999.1671.1619726944', '1.3.6.1.4.1.53684.1.1.2.1887816670.2028.1635775642.129977', '1.2.840.113711.999999.21610.1646414415', '1.3.51.0.1.1.192.168.153.65.2252238.2252213', '2.16.840.1.114151.3386750771734545678043328774845832515830211116', '1.2.528.1.1001.100.2.4193.4669.112678817.20210422153944554', '2.16.840.1.114151.2.2.1.43631.4650.31951486', '1.3.6.1.4.1.53684.1.1.2.1671342546.10164.1628614217.180319', '1.2.840.113711.999999.24317.1621439720', '2.16.840.1.114151.2285048308964144032341578906686371549430210809', '1.3.12.2.1107.5.8.15.132618.30000021110910373877900000013', '1.2.528.1.1003.1.12221815125611000473843013372778.2', '1.3.6.1.4.1.53684.1.1.2.1671342546.2188.1628108423.494577', '1.2.840.113845.11.1000000001785349915.20200529104851.1152983', '1.3.6.1.4.1.53684.1.1.2.3897358757.8788.1611263570.280060', '1.3.12.2.1107.5.8.15.132618.30000021091608245681600001832', '1.2.840.113711.999999.30491.1631104700', '2.16.840.1.114151.2041970537448264662487100815569651061270230111', '1.2.276.0.7230010.3.1.2.3300849612.1684.1661442600.631763', '1.2.840.113845.11.1000000001785349915.20221229064020.2529705', '2.16.840.1.114151.2735768417436547766506548772841937588470180604', '1.3.6.1.4.1.53684.1.1.2.1887816670.2028.1635951933.431316', '1.3.6.1.4.1.53684.1.1.2.1887816670.3036.1633631332.443957', '1.2.840.113845.11.1000000001785349915.20120402110245.5893744', '2.16.840.1.114151.4.25.42092.3959.10219013', '1.2.840.114350.2.17.2.798268.2.217792231.1', '1.2.528.1.1001.100.2.8577.7557.112678817.20210317194329563', '1.3.6.1.4.1.53684.1.1.2.3897358757.1880.1610391974.216594', '1.3.6.1.4.1.53684.1.1.2.3897358757.1868.1622564894.269804', '2.16.840.1.114151.433600830058172882661864142587832193270180426', '1.3.6.1.4.1.53684.1.1.2.1671342546.8364.1626120644.1109885', '1.2.840.113619.2.182.101241254137204.1612884228.2304615', '1.3.6.1.4.1.53684.1.1.2.1887816670.1780.1680885784.934713', '1.3.6.1.4.1.53684.1.1.2.3897358757.1880.1610388577.210643', '1.3.6.1.4.1.53684.1.1.2.1671342546.8364.1626208909.1245029', '1.2.528.1.1003.2.77634651672400530875495810153903.2', '1.2.528.1.1003.2.31775500371413389175804056896420.2', '1.3.6.1.4.1.53684.1.1.2.1671342546.8364.1626373324.1564789', '1.3.6.1.4.1.53684.1.1.2.3897358757.9784.1618591733.32281', '2.16.840.1.114151.1922780271092548282680527787390129557750190326', '1.3.6.1.4.1.53684.1.1.2.3897358757.6976.1619799471.18041', '1.2.840.113845.11.1000000001785349915.20211005163517.1828849', '2.16.840.1.114151.871295696367131567951153063849067434230220506', '1.3.12.2.1107.5.8.15.132618.30000021122210304997300000013', '1.3.6.1.4.1.53684.1.1.2.1442689931.1360.1688663994.872', '1.2.840.113845.11.1000000001785349915.20230119121046.2561314', '1.3.6.1.4.1.53684.1.1.2.1887816670.1984.1668095103.1125598', '1.2.840.113845.11.1000000001951524609.20210317110644.5629605', '1.3.6.1.4.1.53684.1.1.2.1887816670.12640.1665583636.377954', '1.2.528.1.1003.1.12313615334485000449186714846763.2', '1.3.6.1.4.1.53684.1.1.2.1671342546.2188.1627999401.348944', '1.3.6.1.4.1.53684.1.1.2.1887816670.2028.1636484051.912659', '1.2.840.113711.999999.10480.1617632460', '1.3.6.1.4.1.19291.2.1.1.11287420931131606480787284123137', '1.2.528.1.1001.100.2.5741.5717.112678817.20210312141008391', '1.3.6.1.4.1.53684.1.1.2.1671342546.8948.1625058091.26821', '1.2.840.113711.999999.15329.1622641091', '1.2.840.113745.101000.1111000.43278.6780.23207692', '1.2.840.113696.593053.500.181637.20141010085458', '1.3.6.1.4.1.53684.1.1.2.1887816670.8464.1628526218.935', '1.2.528.1.1001.100.2.8093.8097.112678817.20210315171805443', '1.3.12.2.1107.5.8.15.132618.30000022021112265816500000010', '1.2.528.1.1003.2.10972157549161692743411631714274.2', '1.3.6.1.4.1.53684.1.1.2.1671342546.8364.1626703500.1772999', '1.2.528.1.1001.100.2.11073.11077.112678817.20210204152607721', '1.2.840.113711.999999.22178.1626305730', '1.2.528.1.1003.1.12301502881741000056166814777649.2', '1.2.528.1.1001.100.2.4193.4669.112678817.20210422154611295', '2.16.840.1.113669.632.21.347185454.347185454.37462271132136670.6', '1.2.840.113619.2.182.12819322223169136.1337169864.91979', '1.2.840.113711.999999.8344.1628539156', '1.2.840.113711.999999.22637.1625683740', '1.3.6.1.4.1.53684.1.1.2.1671342546.8128.1621009652.168128', '1.3.12.2.1107.5.8.15.132618.30000021081210284034400001506', '1.2.840.113845.11.1000000001785349915.20190724135922.9740177', '1.3.6.1.4.1.53684.1.1.2.3897358757.8788.1611152477.158923', '1.3.6.1.4.1.53684.1.1.2.3897358757.8788.1611151580.155334', '1.3.12.2.1107.5.8.15.132618.30000021050614384238100000010', '1.3.6.1.4.1.53684.1.1.2.3897358757.1976.1625589694.622039', '1.2.528.1.1001.100.2.5513.6577.112678817.20210511152625005', '1.3.6.1.4.1.53684.1.1.2.1671342546.10164.1628534078.114379', '1.3.6.1.4.1.53684.1.1.2.1671342546.12196.1632339553.46284', '1.2.840.114350.2.277.2.798268.2.526374643.1', '1.2.528.1.1001.100.2.7045.6493.112678817.20210218185510556', '1.2.840.114350.2.17.2.798268.2.147576452.1', '1.3.6.1.4.1.53684.1.1.2.1477520070.3364.1701292534.52064', '1.2.840.113711.5120.13.6032.627559796.26.2116281012.11690', '1.2.840.113711.999999.14751.1620222633', '1.3.6.1.4.1.53684.1.1.2.3897358757.1868.1622572927.289647', '1.3.6.1.4.1.53684.1.1.2.1671342546.8948.1625153324.339890', '1.3.6.1.4.1.53684.1.1.2.3897358757.6976.1620223517.354231', '1.3.6.1.4.1.53684.1.1.2.3897358757.4924.1619454671.198042', '1.2.124.113532.10.17.130.37.20161021.73619.27840008', '1.3.6.1.4.1.53684.1.1.2.1671342546.8128.1621013441.201320', '1.2.528.1.1001.100.2.7025.4701.112678817.20210125135633781', '1.2.840.113711.999999.2816.1627333340', '1.2.840.113711.999999.3413.1617630296', '1.2.840.113711.999999.8237.1631810325', '1.2.528.1.1001.100.2.9041.9045.112678817.20210408144453094', '1.2.840.113711.999999.27202.1620427133', '2.16.840.1.114151.2752484654525752485615112611316356501750171215', '1.3.6.1.4.1.53684.1.1.2.1671342546.8364.1625581437.29140', '1.3.6.1.4.1.53684.1.1.2.3897358757.8788.1611259151.265256', '1.2.528.1.1003.1.12229670508733000056166814777649.2', '1.3.12.2.1107.5.8.15.132618.30000021092913035970100000013', '1.3.12.2.1107.5.8.15.132618.30000021050609100180000000010', '1.2.840.113745.101000.1111000.44193.6190.43643038', '1.3.6.1.4.1.53684.1.1.2.1671342546.2552.1631649228.16334', '1.3.12.2.1107.5.8.15.132618.30000022012712592670400000010', '2.16.840.1.114151.4.25.41343.3937.8119459', '1.3.6.1.4.1.53684.1.1.2.1671342546.8948.1625576022.661385', '1.2.840.114350.2.277.2.798268.2.561458620.1', '1.3.6.1.4.1.53684.1.1.2.3897358757.5956.1621344699.557055']
print(len(done), 'patients already done')
patients = [pat.split('/')[-1].split('_')[-1].split('.csv')[0] for pat in glob.glob(f'{path}/*.csv')]
print(len(patients), 'patients found')
patients = sorted(list(set(patients) - set(done)))
print(len(patients), 'patients found')

for patient in ['1.2.528.1.1003.1.12306874907342000056166814777649.2',
                '2.16.840.1.114151.1040466990330684466720442202860436476150210802',
                '2.16.840.1.114151.1922780271092548282680527787390129557750190326'][:]:
    # if not os.path.exists(f'{output_path}/images/{patient}.nii.gz'):
    print(patient)
    df = pd.read_csv(f'{path}/saxdf___{patient}.csv')

    image = load_nii(f'{path}/image___{patient}.nii.gz')
    mask = load_nii(f'{path}/masks___{patient}.nii.gz').astype('uint8')



    if df['true_slicelocation'].values[-1] < df['true_slicelocation'].values[0]:
        image = image[:,:,::-1,:]
        mask = mask[:,:,::-1,:]

    x_min, y_min, x_max, y_max = find_crop_box(mask, crop_factor = 2)

    image = image[y_min:y_max,x_min:x_max,...]
    mask = mask[y_min:y_max,x_min:x_max,...]

    image = pad_to_square(image)
    mask = pad_to_square(mask)

    target_image = 128
    target_time = 32
    time_zoom = target_time / image.shape[3]
    image_zoom = 1 #target_image / image.shape[0]

    image = zoom(image, (image_zoom, image_zoom, 1, time_zoom))
    mask = zoom(mask, (image_zoom, image_zoom, 1, time_zoom), order=0)


    nib_image = nib.Nifti1Image(image, affine=np.eye(4))
    nib.save(nib_image, f'{output_path}/images/{patient}.nii.gz')

    nib_mask = nib.Nifti1Image(mask, affine=np.eye(4), dtype=np.uint8)
    nib.save(nib_mask, f'{output_path}/masks/{patient}.nii.gz') 


    image = load_nii(f'{output_path}/images/{patient}.nii.gz')
    mask = load_nii(f'{output_path}/masks/{patient}.nii.gz').astype('uint8')

    # make_2d_video(image, np.eye(3)[mask] , patient)
    # make_3d_video(np.eye(3)[mask], patient)

146 patients already done
227 patients found
81 patients found
(94, 94, 12, 32)
(194, 194, 14, 32)
(210, 210, 14, 32)


In [3]:
import nibabel as nib
import matplotlib.pyplot as plt

import matplotlib.tight_bbox as tight_bbox
import matplotlib.animation as animation
import matplotlib.pylab as plt

from IPython.display import HTML
from scipy.ndimage import binary_dilation, binary_erosion
from scipy.ndimage import zoom

import numpy as np
import pandas as pd
import trimesh
import glob
import os



def load_nii(nii_path):
    file = nib.load(nii_path)
    data = file.get_fdata()
    return data


# Load the NIfTI file
path = 'cine_sax_compressed_derivatives'
output_path = 'clean'
patients = [pat.split('/')[-1].split('_')[-1].split('.csv')[0] for pat in glob.glob(f'{path}/*.csv')]

for patient in patients[:]:


    image = load_nii(f'{output_path}/images/{patient}.nii.gz')
    mask = load_nii(f'{output_path}/masks/{patient}.nii.gz').astype('uint8')

    print(image.shape, mask.shape)
    if image.shape[2] != mask.shape[2]:
        print(patient)

ModuleNotFoundError: No module named 'matplotlib.tight_bbox'

(128, 128, 12, 32)